<a href="https://colab.research.google.com/github/romerofernandes/class_fault/blob/main/class_fault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação e análise

In [31]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

In [4]:
df_fault = pd.read_csv('/content/classData.csv')

In [36]:
df_fault_test = pd.read_csv('/content/detect_dataset.csv')
df_fault_test.drop(columns = ['Unnamed: 7', 'Unnamed: 8'], inplace =True)

In [37]:
df_fault_test.head()

,Output (S),Ia,Ib,Ic,Va,Vb,Vc
0,0,-170.472196,9.219613,161.252583,0.054490,-0.659921,0.605431
1,0,-122.235754,6.168667,116.067087,0.102000,-0.628612,0.526202
2,0,-90.161474,3.813632,86.347841,0.141026,-0.605277,0.464251
3,0,-79.904916,2.398803,77.506112,0.156272,-0.602235,0.445963
4,0,-63.885255,0.590667,63.294587,0.180451,-0.591501,0.411050


[0 0 0 0] - No Fault

[1 0 0 1] - LG fault (Between Phase A and Gnd)

[0 0 1 1] - LL fault (Between Phase A and Phase B)

[1 0 1 1] - LLG Fault (Between Phases A,B and ground)

[0 1 1 1] - LLL Fault(Between all three phases)

[1 1 1 1] - LLLG fault( Three phase symmetrical fault)

In [15]:
df_fault.head()

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,cod_fault,fault
0,1,0,0,1,-151.291812,-9.677452,85.800162,0.400750,-0.132935,-0.267815,1001,1
1,1,0,0,1,-336.186183,-76.283262,18.328897,0.312732,-0.123633,-0.189099,1001,1
2,1,0,0,1,-502.891583,-174.648023,-80.924663,0.265728,-0.114301,-0.151428,1001,1
3,1,0,0,1,-593.941905,-217.703359,-124.891924,0.235511,-0.104940,-0.130570,1001,1
4,1,0,0,1,-643.663617,-224.159427,-132.282815,0.209537,-0.095554,-0.113983,1001,1


In [6]:
df_fault.dtypes

G       int64
C       int64
B       int64
A       int64
Ia    float64
Ib    float64
Ic    float64
Va    float64
Vb    float64
Vc    float64
dtype: object

In [7]:
df_fault['cod_fault'] = df_fault.G.astype(str) + df_fault.C.astype(str) + df_fault.B.astype(str) + df_fault.A.astype(str)

In [8]:
df_fault['fault'] = df_fault.apply(lambda x: 0 if x.cod_fault == '0000' else 1, axis=1)

In [9]:
df_fault_vis = df_fault.groupby('cod_fault').agg({'G':'count'}).reset_index()

In [10]:
df_fault_vis.cod_fault.unique()

array(['0000', '0110', '0111', '1001', '1011', '1111'], dtype=object)

In [11]:
df_fault_vis

,cod_fault,G
0,0000,2365
1,0110,1004
2,0111,1096
3,1001,1129
4,1011,1134
5,1111,1133


In [12]:
df_fault.groupby('fault').agg({'G':'count'}).reset_index()

,fault,G
0,0,2365
1,1,5496


##Treinamento e teste

In [16]:
df_fault.head()

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,cod_fault,fault
0,1,0,0,1,-151.291812,-9.677452,85.800162,0.400750,-0.132935,-0.267815,1001,1
1,1,0,0,1,-336.186183,-76.283262,18.328897,0.312732,-0.123633,-0.189099,1001,1
2,1,0,0,1,-502.891583,-174.648023,-80.924663,0.265728,-0.114301,-0.151428,1001,1
3,1,0,0,1,-593.941905,-217.703359,-124.891924,0.235511,-0.104940,-0.130570,1001,1
4,1,0,0,1,-643.663617,-224.159427,-132.282815,0.209537,-0.095554,-0.113983,1001,1


In [22]:
X = df_fault[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc']]
y = df_fault['fault']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [25]:
classif_fault = RandomForestClassifier()

In [26]:
classif_fault.fit(X_train, y_train)

RandomForestClassifier()

In [27]:
y_predict = classif_fault.predict( X_test)

In [29]:
confusion_matrix(y_test, y_predict)

array([[ 464,    1],
       [   0, 1108]])

In [33]:
precision_score(y_test, y_predict, average=None)

array([1.        , 0.99909829])

In [38]:
y_predict2 = classif_fault.predict(df_fault_test[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc']])

In [39]:
confusion_matrix(df_fault_test['Output (S)'], y_predict2)

array([[4649, 1856],
       [   0, 5496]])

In [40]:
precision_score(df_fault_test['Output (S)'], y_predict2, average=None)

array([1.        , 0.74755169])